In [25]:
# Parameters
NUM_AGENTS = 5
NUM_MEETINGS = 15
AGENT_SKILLS = ['fire_cert', 'bodyguard_cert', 'first_aid', 'security_clearance']
NIGHT_SHIFTS_PER_AGENT_PER_WEEK = 1
AGENT_HOURS_PER_WEEK = 40
MEETING_DURATION_OPTIONS = [1, 2, 3]  # In hours


In [26]:
import random
import folium
import json

In [27]:
# Define the zones with names for clarity
zones = [
    {
        'name': 'GENEVA1',
        'sw': (46.160963, 6.084271),
        'ne': (46.192016, 6.138650)
    },  # GENEVA1
    {
        'name': 'GENEVA2',
        'sw': (46.213758, 6.059573),
        'ne': (46.235041, 6.144283)
    },  # GENEVA2
    {
        'name': 'GENEVA3',
        'sw': (46.190981, 6.144247),
        'ne': (46.205087, 6.183324)
    },  # GENEVA3
    {
        'name': 'GENEVA4',
        'sw': (46.160963, 6.0),
        'ne': (46.192016, 6.06)
    }   # GENEVA4
]

# Define weights corresponding to the zones
# GENEVA1: 40%, GENEVA3: 40%, GENEVA2: 15%, GENEVA4: 5%
zone_weights = [30, 20, 30, 20]

In [28]:
def generate_random_location(zones, weights):
    """
    Selects a random zone based on the provided weights and generates a random location within that zone.
    
    :param zones: List of zone dictionaries with 'sw' and 'ne' coordinates.
    :param weights: List of weights corresponding to each zone.
    :return: Tuple of (latitude, longitude)
    """
    zone = random.choices(zones, weights=weights, k=1)[0]
    lat = random.uniform(zone['sw'][0], zone['ne'][0])
    lon = random.uniform(zone['sw'][1], zone['ne'][1])
    return (lat, lon)

In [29]:
def generate_agents(num_agents):
    agents = []
    for i in range(num_agents):
        agent = {
            'agent_id': f'Agent_{i+1}',
            'skills': random.sample(AGENT_SKILLS, random.randint(1, len(AGENT_SKILLS))),
            'home_location': generate_random_location(zones, zone_weights),
            'night_shifts_per_week': NIGHT_SHIFTS_PER_AGENT_PER_WEEK,
            'weekly_hours': AGENT_HOURS_PER_WEEK
        }
        agents.append(agent)
    return agents

In [30]:
def generate_meetings(num_meetings):
    meetings = []
    for i in range(num_meetings):
        meeting_duration = random.choice(MEETING_DURATION_OPTIONS)
        start_hour = random.randint(8, 16)  # Ensure there's enough time for duration
        end_hour = start_hour + meeting_duration
        if end_hour > 18:
            end_hour = 18
            start_hour = end_hour - meeting_duration
        meeting = {
            'meeting_id': f'Meeting_{i+1}',
            'required_skill': random.choice(AGENT_SKILLS),
            'location': generate_random_location(zones, zone_weights),
            'time_window': (
                start_hour,  # Start hour
                end_hour     # End hour
            ),
            'duration': meeting_duration
        }
        meetings.append(meeting)
    return meetings

In [31]:
# Initialize agents and meetings
agents = generate_agents(NUM_AGENTS)
meetings = generate_meetings(NUM_MEETINGS)

In [32]:
# Create a map centered around Geneva
map_geneva = folium.Map(location=[46.2044, 6.1432], zoom_start=13)

In [33]:
# Add agent home locations
for agent in agents:
    folium.Marker(
        location=agent['home_location'],
        popup=f"{agent['agent_id']} Skills: {', '.join(agent['skills'])}",
        icon=folium.Icon(color='blue', icon='user')
    ).add_to(map_geneva)

In [34]:
# Add meeting locations
for meeting in meetings:
    folium.Marker(
        location=meeting['location'],
        popup=f"{meeting['meeting_id']} Required Skill: {meeting['required_skill']}",
        icon=folium.Icon(color='red', icon='briefcase')
    ).add_to(map_geneva)

In [35]:
# Display the map
map_geneva


In [36]:
# Prepare data for Timefold
timefold_data = {
    'vehicles': [
        {
            'name': agent['agent_id'],
            'homeLocation': agent['home_location'],
            'capacity': agent['weekly_hours'],
            'skills': agent['skills']
        }
        for agent in agents
    ],
    'visits': [
        {
            'name': meeting['meeting_id'],
            'location': meeting['location'],
            'required_skill': meeting['required_skill'],
            'time_window': meeting['time_window'],
            'duration': meeting['duration']
        }
        for meeting in meetings
    ]
}

# Save to JSON file with indentation
with open('timefold-data.json', 'w') as f:
    json.dump(timefold_data, f, indent=4)  # Added indent parameter